In [1]:
"""
# Description:  Converts the U.S. Department of Interior's National 
#               Fire Plan Operations and Reporting System (NFPORS) dataset 
#               into the Task Force standardized schema.  Dataset
#               is enriched with vegetation, ownership, county, WUI, 
#               Task Force Region, and year.
#               BLM and NPS current year treatments are inlcuded but past 
#               years treatments excluded and managed by seperate notebooks.
# Author: Spatial Informatics Group LLC
# Version: 1.0.0
# Date Created: Jan 24, 2024
"""
import os
import arcpy
import datetime
from scripts.utils import init_gdb, check_exists, og_file_input
from scripts._3_NFPORS import Model_NFPORS

workspace, scratch_workspace = init_gdb()

date_id = datetime.datetime.now().strftime("%Y-%m-%d").replace('-','') # like 20221216
# arcpy.env.overwriteOutput = True

# testing hash replacement. kdw
# INPUTS
# Will need to re-download NFPORS dataset each for each update from the below web service URL
# TODO integrate Download Service script tool for NFPORS because the service is throttled to limit the number of downloadable features per call 
# https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/15

nfpors_poly = og_file_input(prefix = 'nfpors_fuels_treatments_', filetype = 'Polygon', gdb = os.path.join(workspace, "a_Originals"))
input_original_polys = os.path.join(workspace,'a_Originals',nfpors_poly)

# Downloads and saves BIA and FWS current year project points to scratch gdb prior to being appended together in Model_NFPORS
query_CA = "statename='California'"

bia_api = 'https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/1'
bia_fs = arcpy.FeatureSet(bia_api, query_CA)
input_original_pts_BIA = os.path.join(scratch_workspace, f'NFPORSCurrentFYTreatmentsBIA_{date_id}')
get_bia_pt = arcpy.CopyFeatures_management(bia_fs, input_original_pts_BIA)
print("BIA current year points copied to scratch")

fws_api = 'https://usgs.nfpors.gov/arcgis/rest/services/nfpors_WM/MapServer/5'
fws_fs = arcpy.FeatureSet(fws_api, query_CA)
input_original_pts_FWS = os.path.join(scratch_workspace, f'NFPORSCurrentFYTreatmentsFWS_{date_id}')
get_fws_pt = arcpy.CopyFeatures_management(fws_fs, input_original_pts_FWS)
print("FWS current year points copied to scratch")

# OUTPUTS
# these are setup to output new 'timestamped' outputs

output_original_polys = os.path.join(workspace,'a_Originals',f'nfpors_current_fy_{date_id}') # created from aggregating input_original_pts_BIA and input_original_pts_FWS 
output_pts_standardized = os.path.join(workspace,'c_Standardized',f'nfpors_fuels_treatments_pts_standardized_{date_id}')
output_pts_enriched = os.path.join(workspace, 'd_Enriched',f'nfpors_fuels_treatments_pts_enriched_{date_id}')
output_polys_standardized = os.path.join(workspace,'c_Standardized',f'nfpors_fuels_treatments_polys_standardized_{date_id}')
output_polys_enriched = os.path.join(workspace,'d_Enriched',f'nfpors_fuels_treatments_polys_enriched_{date_id}')

BIA current year points copied to scratch
FWS current year points copied to scratch


In [2]:
Model_NFPORS(input_original_polys,
    input_original_pts_BIA, 
    input_original_pts_FWS, 
    output_original_polys, 
    output_polys_standardized, 
    output_polys_enriched, 
    output_pts_standardized, 
    output_pts_enriched
)

Start Time Thu Jan  4 09:18:31 2024
Performing Polygons Standardization
Saving Output Polys Standardized: c:\Users\sageg\source\repos\mas-python\PC414 CWI Million Acres.gdb\c_Standardized\nfpors_fuels_treatments_polys_standardized_20240104
Performing Polygon Enrichments
Executing Polygon Enrichments...
   Calculating Broad Vegetation Type...
     step 1/34 summarize veg within polygons
     step 2/34 summarize attributes
     step 3/34 add join
     step 4/34 convert table to table
     step 5/34 delete identical
     step 6/34 add join
     step 7/34 select layer by attribute
     step 8/34 calculate user defined veg field yes
     step 9/34 select layer by attribute
     step 10/34 calculate veg domain code
     step 11/34 calculate user defined veg field no
     step 12/34 remove join
   Calculating WUI...
     step 13/34 select layer by attribute
     step 14/34 select layer by WUI location
     step 15/34 calculate WUI yes
     step 16/34 select layer by attribute
     step 17/34 